# Imports

In [ ]:
import numpy as np

from matplotlib import pyplot as plt
from src.environments.mg_simple import MGSimple
from src.rl.a2c.c_mg_simple import set_all_seeds

# Utils

In [ ]:
def plot_results(env, states, rewards, actions, net_energy, title, save=False, filename='results.png'):

    # Get list indexes

    quart_index = int(len(states) / 4)
    mid_index = int(len(states)/2)
    three_quart_index = int(len(states) * 3/4)
    last_index = len(states) - 1

    # Parse list of arrays

    all_rewards = np.stack(rewards, axis=0)
    all_actions = np.stack(actions, axis=0)
    all_states = np.stack(states, axis=0)
    all_net_energy = np.stack(net_energy, axis=0)

    all_socs = all_states[:,:,:,7]

    # Plot results of interest

    fig = plt.figure(figsize=(10, 15))
    axs = fig.subplots(5, 2)

    for ax_x in axs:
        for ax_y in ax_x:
            ax_y.minorticks_on()
            ax_y.grid(True, which='both', axis='both', alpha=0.5)

    axs[0][0].plot(all_rewards[0,:,:].mean(axis=1), label='0%')
    axs[0][0].plot(all_rewards[quart_index,:,:].mean(axis=1), label='25%')
    axs[0][0].plot(all_rewards[mid_index,:,:].mean(axis=1), label='50%')
    axs[0][0].plot(all_rewards[three_quart_index,:,:].mean(axis=1), label='75%')
    axs[0][0].plot(all_rewards[last_index,:,:].mean(axis=1), label='100%')
    axs[0][0].set_title('Mean reward through time')
    axs[0][0].legend()

    axs[0][1].plot(all_rewards.sum(axis=1).mean(axis=1), label='Reward')
    axs[0][1].set_title('Mean reward through epochs')
    axs[0][1].legend()

    axs[1][0].plot(all_actions[0, :, :].mean(axis=1), label='0%')
    axs[1][0].plot(all_actions[quart_index, :, :].mean(axis=1), label='25%')
    axs[1][0].plot(all_actions[mid_index, :, :].mean(axis=1), label='50%')
    axs[1][0].plot(all_actions[three_quart_index, :, :].mean(axis=1), label='75%')
    axs[1][0].plot(all_actions[last_index, :, :].mean(axis=1), label='100%')
    axs[1][0].set_title('Mean action through time')
    axs[1][0].legend()

    axs[1][1].plot(all_actions.mean(axis=1).mean(axis=-1), label='Action')
    axs[1][1].legend()
    axs[1][1].set_title('Mean action through epochs')

    axs[2][0].plot(all_socs[0, :, :].mean(axis=1), label='0%')
    axs[2][0].plot(all_socs[quart_index, :, :].mean(axis=1), label='25%')
    axs[2][0].plot(all_socs[mid_index, :, :].mean(axis=1), label='50%')
    axs[2][0].plot(all_socs[three_quart_index, :, :].mean(axis=1), label='75%')
    axs[2][0].plot(all_socs[last_index, :, :].mean(axis=1), label='100%')
    axs[2][0].legend()
    axs[2][0].set_title('Mean SOC through time')

    axs[2][1].plot(all_socs.mean(axis=1).mean(axis=-1), label='SOC')
    axs[2][1].set_title('Mean SOC through epochs')
    axs[2][1].legend()

    axs[3][0].plot(env.mg.pv_gen, label='PV')
    axs[3][0].plot(env.mg.demand, label='Demand')
    axs[3][0].set_title('PV and Demand')
    axs[3][0].legend()

    axs[3][1].plot(env.mg.price, label='Price')
    axs[3][1].plot(env.mg.emission, label='Emission factor')
    axs[3][1].set_title('Price and Emission factor')
    axs[3][1].legend()

    axs[4][0].plot(all_net_energy[0, :, :].mean(axis=0), label='0%')
    axs[4][0].plot(all_net_energy[quart_index, :, :].mean(axis=0), label='25%')
    axs[4][0].plot(all_net_energy[mid_index, :, :].mean(axis=0), label='50%')
    axs[4][0].plot(all_net_energy[three_quart_index, :, :].mean(axis=0), label='75%')
    axs[4][0].plot(all_net_energy[last_index, :, :].mean(axis=0), label='100%')
    axs[4][0].plot((env.mg.demand - env.mg.pv_gen), label='Remaining', linestyle='--', linewidth=2)
    axs[4][0].legend()
    axs[4][0].set_title('Mean net energy through time')

    axs[4][1].plot(all_net_energy.mean(axis=1).sum(axis=-1), label='Net energy')
    axs[4][1].set_title('Mean net energy through epochs')
    axs[4][1].legend()

    fig.suptitle(title)
    fig.tight_layout()

    if save:
        plt.savefig(filename)

    plt.grid()
    plt.show()


## Zero agent

In [ ]:
set_all_seeds(0)
batch_size = 1
env = MGSimple(batch_size=batch_size, steps=24, min_temp=29, max_temp=31, peak_pv_gen=1, peak_grid_gen=1, peak_load=1, disable_noise=True)

# Initialize states and rewards

state_0, r_0, done, _ = env.reset()

states_za, rewards_za, actions_za = [], [], []
states_za.append(state_0)
rewards_za.append(r_0)
while not done:

    action = np.zeros((batch_size, 1))

    s_t, r_t, done, _ = env.step(action)

    states_za.append(s_t)
    rewards_za.append(r_t)
    actions_za.append(action)

states_za = np.stack(states_za, axis=1)
rewards_za = np.stack(rewards_za, axis=1)
actions_za = np.stack(actions_za, axis=1)
rewards_za.sum()

In [ ]:
rewards_za.squeeze()

In [ ]:
states_za.squeeze()[:,(0,2,3,4,5)]

In [ ]:
plt.plot(rewards_za.squeeze())
plt.plot(actions_za.squeeze())

# Random Agent

In [ ]:
set_all_seeds(0)
batch_size = 1
env = MGSimple(batch_size=batch_size, steps=24, min_temp=29, max_temp=31, peak_pv_gen=1, peak_grid_gen=1, peak_load=1, disable_noise=True)

# Initialize states and rewards

state_0, r_0, done, _ = env.reset()

states_ra, rewards_ra, actions_ra = [], [], []

while not done:

    action = np.random.uniform(low=-1, high=1, size=(batch_size, 1))

    s_t, r_t, done, _ = env.step(action)

    states_ra.append(s_t)
    rewards_ra.append(r_t)
    actions_ra.append(action)

states_ra = np.stack(states_ra, axis=1)
rewards_ra = np.stack(rewards_ra, axis=1)
actions_ra = np.stack(actions_ra, axis=1)
rewards_ra.sum()

In [ ]:
np.random.uniform(low=-1, high=1, size=(batch_size, 1)).shape

# CVX Agent

In [ ]:
set_all_seeds(0)
batch_size = 1
env = MGSimple(batch_size=batch_size, steps=24, min_temp=29, max_temp=31, peak_pv_gen=1, peak_grid_gen=1, peak_load=1, disable_noise=True)

# Initialize states and rewards

s_t, r_t, done, _ = env.reset()

states_ra, rewards_ra, actions_ra = [], [], []

perf_actions = [ 1.87618149e-01,  1.16353135e-01,  8.45745864e-02,  7.83851802e-02,
         9.81474354e-02, -4.22038145e-02, -2.47501938e-01, -2.75372733e-01,
         5.67474161e-01,  2.32525839e-01, -2.68996106e-01, -1.30855062e-01,
        -1.30862054e-01, -2.69286778e-01,  1.91850189e-01,  5.15841571e-01,
         9.23082401e-02,  1.19832551e-10, -2.12988428e-01, -3.68287674e-01,
        -2.18723898e-01,  1.26685365e-01,  2.85300459e-01,  3.49321574e-01]
         
while not done:

    action = np.ones((batch_size, 1))
    action = action * perf_actions[int(s_t[0,0])]

    s_t, r_t, done, _ = env.step(action)

    states_ra.append(s_t)
    rewards_ra.append(r_t)
    actions_ra.append(action)

states_ra = np.stack(states_ra, axis=1)
rewards_ra = np.stack(rewards_ra, axis=1)
actions_ra = np.stack(actions_ra, axis=1)
rewards_ra.sum()

In [ ]:
states_ra[0,:,2]
# states_ra[0,:,7]#SOC

In [ ]:
states_ra.shape, rewards_ra.shape,actions_ra.shape


# A2C Causality cont.

In [ ]:
import traceback

from src.environments.mg_simple import MGSimple
from src.rl.a2c.c_mg_simple import Agent

# Get arguments from command line

disable_logging = False
disable_noise = True
batch_size = 8
training_steps = 1500
rollout_steps = 24
actor_lr = 0.0001
critic_lr = 0.001
actor_nn = 64
critic_nn = 64
gamma = 1
enable_gpu = True
central_agent = False # Not needed yet
random_soc_0 = False
encoding = False # Not needed yet
extended_observation = False # Not needed yet
epsilon = 0.001

# Start wandb logger

try:

    '''
        Setup all the configurations for Wandb
    '''

    wdb_config={
        "training_steps": training_steps,
        "batch_size": batch_size,
        "rollout_steps": rollout_steps,
        "agent_actor_lr": actor_lr,
        "agent_critic_lr": critic_lr,
        "agent_actor_nn": actor_nn,
        "agent_critic_nn": critic_nn,
        "gamma": gamma,
        "central_agent": central_agent,
        "random_soc_0": random_soc_0,
        "encoding": encoding,
        "extended_observation": extended_observation,
        "epsilon": epsilon,
    }

    '''
        Run the simulator
    '''

    set_all_seeds(0)

    # Instantiate the environment

    my_env = MGSimple(
        batch_size=batch_size, steps = rollout_steps, min_temp = 29, max_temp = 31, peak_pv_gen = 1, peak_grid_gen = 1, peak_load = 1,
        random_soc_0=random_soc_0, disable_noise=disable_noise
    )

    # Instantiate the agent

    agent = Agent(
        env=my_env, critic_lr=critic_lr, actor_lr=actor_lr, actor_nn=actor_nn, critic_nn=critic_nn, batch_size=batch_size, gamma=gamma,
        extended_obs=extended_observation, wandb_dict=wdb_config, enable_gpu=enable_gpu, disable_wandb=disable_logging,
    )

    # Launch the training

    all_states, all_rewards, all_actions = agent.train(training_steps=training_steps, epsilon=epsilon)

    # Finish Wandb execution

    agent.wdb_logger.finish()

except (RuntimeError, KeyboardInterrupt):

    traceback.print_exc()

# A2C Causality disc.

In [ ]:
import traceback

from src.environments.mg_simple import MGSimple
from src.rl.a2c.d_mg_simple import Agent, set_all_seeds

# Get arguments from command line

disable_logging = True
disable_noise = True
batch_size = 64
training_steps = 1500
rollout_steps = 24
actor_lr = 0.00065
critic_lr = 0.0001
actor_nn = 256
critic_nn = 64
gamma = 1
enable_gpu = True
central_agent = False # Not needed yet
random_soc_0 = False
encoding = False # Not needed yet
extended_observation = False # Not needed yet
epsilon = 0.9

# Start wandb logger

try:

    '''
        Setup all the configurations for Wandb
    '''

    wdb_config={
        "training_steps": training_steps,
        "batch_size": batch_size,
        "rollout_steps": rollout_steps,
        "agent_actor_lr": actor_lr,
        "agent_critic_lr": critic_lr,
        "agent_actor_nn": actor_nn,
        "agent_critic_nn": critic_nn,
        "gamma": gamma,
        "central_agent": central_agent,
        "random_soc_0": random_soc_0,
        "encoding": encoding,
        "extended_observation": extended_observation,
        "epsilon": epsilon,
    }

    '''
        Run the simulator
    '''

    set_all_seeds(0)

    # Instantiate the environment

    my_env = MGSimple(
        batch_size=batch_size, steps = rollout_steps, min_temp = 29, max_temp = 31, peak_pv_gen = 1, peak_grid_gen = 1, peak_load = 1,
        random_soc_0=random_soc_0, disable_noise=disable_noise
    )

    # Instantiate the agent

    agent = Agent(
        env=my_env, critic_lr=critic_lr, actor_lr=actor_lr, actor_nn=actor_nn, critic_nn=critic_nn, batch_size=batch_size, gamma=gamma,
        extended_obs=extended_observation, wandb_dict=wdb_config, enable_gpu=enable_gpu, disable_wandb=disable_logging,
    )

    # Launch the training

    all_states, all_rewards, all_actions, all_net_energy = agent.train(training_steps=training_steps, epsilon=epsilon)

    # Finish Wandb execution

    agent.wdb_logger.finish()

except (RuntimeError, KeyboardInterrupt):

    traceback.print_exc()

print('A2C training completed')

In [ ]:
# Plot results

print(wdb_config)

plot_results(my_env, all_states, all_rewards, all_actions, all_net_energy, 'A2C D results', save=False, filename='a2c_results.png')


In [ ]:
# Plot results

print(wdb_config)

plot_results(my_env, all_states, all_rewards, all_actions, all_net_energy, 'A2C D results', save=False, filename='a2c_results.png')


In [ ]:
# Plot results

print(wdb_config)

plot_results(my_env, all_states, all_rewards, all_actions, all_net_energy, 'A2C D results', save=False, filename='a2c_results.png')


In [ ]:
# Plot results

print(wdb_config)

plot_results(my_env, all_states, all_rewards, all_actions, all_net_energy, 'A2C D results', save=False, filename='a2c_results.png')
